In [15]:
import pandas as pd

In [19]:
file_name = "F1_4_Detailed releases at facility level with E-PRTR Sector and Annex I Activity detail into Air.csv"
data = pd.read_csv(file_name)
data.head()

C:\Users\joaop\AppData\Local\Temp\ipykernel_7636\1090922939.py:2: DtypeWarning: Columns (7,10,16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv(file_name)


,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,facilityNameConfidentialityReason,Longitude,Latitude,addressConfidentialityReason,City,targetRelease,pollutant,emissions,reportingYear,releasesConfidentialityReason
0,Austria,4.0,Chemical industry,4(d),Chemical installations for the production on a...,AT.CAED/9008390392737.FACILITY,Nufarm GmbH & CO KG,NaN,14.336056,48.290943,NaN,Linz,AIR,"1,2-dichloroethane (DCE-1,2)",3300.0,2007,NaN
1,Austria,9.0,Other activities,9(c),Installations for the surface treatment of sub...,AT.CAED/9008390396414.FACILITY,MAGNA STEYR Fahrzeugtechnik AG&Co KG,NaN,15.477778,47.019170,NaN,"Graz,07.Bez.:Liebenau",AIR,Non-methane volatile organic compounds (NMVOC),170000.0,2007,NaN
2,Austria,6.0,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,NaN,15.691833,48.259556,NaN,Unterradlberg,AIR,Carbon dioxide (CO2),228000000.0,2007,NaN
3,Austria,6.0,Paper and wood production and processing,6(b),Industrial plants for the production of paper ...,AT.CAED/9008390397176.FACILITY,Fritz EGGER GmbH & Co. OG,NaN,15.691833,48.259556,NaN,Unterradlberg,AIR,Nitrogen oxides (NOX),311000.0,2007,NaN
4,Austria,3.0,Mineral industry,3(c),Installations for the production of cement cli...,AT.CAED/9008390397220.FACILITY,voestalpine Stahl GmbH,NaN,14.159778,47.807083,NaN,Klaus an der Pyhrnbahn,AIR,Carbon dioxide (CO2),329000000.0,2007,NaN


In [20]:
# now choose only london
data = data[data["City"]== 'London']
data.head()

,countryName,EPRTRSectorCode,eprtrSectorName,EPRTRAnnexIMainActivityCode,EPRTRAnnexIMainActivityLabel,FacilityInspireID,facilityName,facilityNameConfidentialityReason,Longitude,Latitude,addressConfidentialityReason,City,targetRelease,pollutant,emissions,reportingYear,releasesConfidentialityReason
297134,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261731,51.5359,NaN,London,AIR,Hydrochlorofluorocarbons (HCFCs),15.0,2009,NaN
300011,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261731,51.5359,NaN,London,AIR,Hydrochlorofluorocarbons (HCFCs),295.0,2010,NaN
302842,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261732,51.5359,NaN,London,AIR,Hydrochlorofluorocarbons (HCFCs),191.0,2011,NaN
304094,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,GB.EEA/13486.FACILITY,"Tate And Lyle Plc, Tate & Lyle",NaN,0.047184,51.5015,NaN,London,AIR,Carbon dioxide (CO2),106000000.0,2012,NaN
305636,United Kingdom,8.0,Animal and vegetable products from the food an...,8(b),Treatment and processing intended for the prod...,UK.CAED/EW_EA-3678.FACILITY,United Biscuits (UK) Limited T/A Pladis,NaN,-0.261720,51.5359,NaN,London,AIR,Hydrochlorofluorocarbons (HCFCs),16.5,2012,NaN


In [27]:
#select relevant data
data = data.loc[:,["targetRelease","pollutant","emissions","reportingYear"]]
data.sort_values('reportingYear',inplace=True)
data.head()

,targetRelease,pollutant,emissions,reportingYear
297134,AIR,Hydrochlorofluorocarbons (HCFCs),15.0,2009
300011,AIR,Hydrochlorofluorocarbons (HCFCs),295.0,2010
302842,AIR,Hydrochlorofluorocarbons (HCFCs),191.0,2011
304094,AIR,Carbon dioxide (CO2),106000000.0,2012
305636,AIR,Hydrochlorofluorocarbons (HCFCs),16.5,2012


In [28]:
#health index
file_name = "health-index-1.csv"
data = pd.read_csv(file_name)
data.head()

,id,utc_created,utc_updated,country,country_code,indicator_code,indicator,value,year
0,1,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.327,1980
1,2,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.375,1985
2,3,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.440,1990
3,4,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.498,1995
4,5,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,Afghanistan,AFG,72206,HDI: Health index,0.536,2000


In [29]:
# now choose only london
data = data[data["country"]== 'United Kingdom']
data.head()

,id,utc_created,utc_updated,country,country_code,indicator_code,indicator,value,year
793,794,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.823,1980
794,795,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.839,1985
795,796,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.855,1990
796,797,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.871,1995
797,798,2016-01-07 10:10:52.689964,2016-01-07 10:10:52.690139,United Kingdom,GBR,72206,HDI: Health index,0.888,2000


In [38]:
data = data.loc[:,["value","year"]]
data.sort_values('year',inplace=True)
data
#some years missing

,value,year
793,0.823,1980
794,0.839,1985
795,0.855,1990
796,0.871,1995
797,0.888,2000
798,0.908,2005
799,0.912,2006
800,0.916,2007
801,0.922,2009
802,0.924,2010
